In [6]:
import sys
!{sys.executable} -m pip install google-cloud-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 73.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 10.3 MB/s eta 0:00:00


In [7]:
from google.cloud import bigquery


# Create a BigQuery client
client = bigquery.Client(project="degroup11")

In [10]:
from prettytable import PrettyTable
import pandas as pd
from prettytable import PrettyTable

ModuleNotFoundError: No module named 'prettytable'

In [9]:
QUERY = ('SELECT ID, has_spouse, gross_salary, alimony_amount, gross_salary - 12 * alimony_amount AS spendable_income FROM `degroup11.group11dataset.individuals` LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

# Get column names
columns = [field.name for field in query_job.result().schema]

# Create a PrettyTable
table = PrettyTable(columns)

# Add rows to the table
for row in rows:
    table.add_row([row[column] for column in columns])

# Print the table
print(table)

NameError: name 'PrettyTable' is not defined

In [21]:
QUERY = ('''
    SELECT 
    ind.gross_salary,
    ind.alimony_amount,
    ind.has_spouse,
    sp.gross_salary AS sp_gross_salary,
    sp.alimony_amount AS sp_alimony_amount,
    CAST(
        IF(ind.has_spouse = FALSE, 
            ind.gross_salary - 12 * ind.alimony_amount, 
            IF(ind.gross_salary > sp.gross_salary, 
            ind.gross_salary - 12 * ind.alimony_amount + 1/3 * (sp.gross_salary - 12 * sp.alimony_amount),
            sp.gross_salary - 12 * sp.alimony_amount + 1/3 * (ind.gross_salary - 12 * ind.alimony_amount))
        ) AS INT64
    ) AS house_spendable_income
FROM 
    degroup11.group11dataset.individuals ind
LEFT JOIN 
    degroup11.group11dataset.spouse sp
ON 
    ind.spouse_ID = sp.spouse_ID
    ''')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

# Get column names
columns = [field.name for field in query_job.result().schema]

# Create a PrettyTable
table = PrettyTable(columns)

# Add rows to the table
for row in rows:
    table.add_row([row[column] for column in columns])

# Print the table
print(table[:10])

+--------------+----------------+------------+-----------------+-------------------+------------------------+
| gross_salary | alimony_amount | has_spouse | sp_gross_salary | sp_alimony_amount | house_spendable_income |
+--------------+----------------+------------+-----------------+-------------------+------------------------+
|    75526     |       0        |    True    |      59122      |         0         |         95233          |
|    134600    |      1200      |   False    |      101412     |         0         |         120200         |
|    144798    |       0        |    True    |      149733     |         0         |         197999         |
|    150580    |       0        |    True    |      47582      |         0         |         166441         |
|    186763    |       0        |   False    |      181201     |         0         |         186763         |
|    59116     |       0        |    True    |      38986      |         0         |         72111          |
|    18988

In [29]:
table.asDataframe()

AttributeError: asDataframe

In [24]:
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

QUERY = '''
    SELECT gross_salary, gross_salary*0.30/12 * ((1 - POWER(1 + 0.0037, -360)) / 0.0037) AS MortgageValue FROM `degroup11.group11dataset.individuals` LIMIT 10
'''

# Run the query
query_job = client.query(QUERY)
rows = query_job.result()

# Get column names
columns = [field.name for field in query_job.result().schema]

# Print column names
print(" | ".join(columns))

# Print separator line
print("-" * (15 * len(columns) - 1))

# Print rows
for row in rows:
    print(" | ".join([str(row[column]) for column in columns]))


gross_salary | MortgageValue
-----------------------------
75526 | 375283.3580552769
134600 | 668817.8904515038
144798 | 719491.0319583719
150580 | 748221.381457559
186763 | 928012.1521128841
59116 | 293743.2274289086
189883 | 943515.2116835281
157499 | 782601.403627202
171667 | 853001.1946518448
168304 | 836290.6852492562


In [23]:
QUERY = ('SELECT gross_salary, gross_salary*0.30/12 * ((1 - POWER(1 + 0.0037, -360)) / 0.0037) AS MortgageValue FROM `degroup11.group11dataset.individuals` LIMIT 10')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)


BadRequest: 400 Unrecognized name: MAX_MONTHLY_MORTGAGE at [5:9]

Location: US
Job ID: 37f83a0f-abaa-4979-b19c-610a0537fed0


In [5]:
# Perform a query.
QUERY = (
    'SELECT * FROM `degroup11.group11dataset.individuals` LIMIT 100')   # use the correct project id, etc.
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

Row((106, 18, True, 106, 75526, False, 0, True, 9288, False, 0), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 'student_loan_amount': 6, 'has_general_loan': 7, 'general_loan_amount': 8, 'has_alimony': 9, 'alimony_amount': 10})
Row((153, 18, False, 153, 134600, False, 0, True, 5314, True, 1200), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 'student_loan_amount': 6, 'has_general_loan': 7, 'general_loan_amount': 8, 'has_alimony': 9, 'alimony_amount': 10})
Row((211, 18, True, 211, 144798, True, 26010, True, 1815, False, 0), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 'student_loan_amount': 6, 'has_general_loan': 7, 'general_loan_amount': 8, 'has_alimony': 9, 'alimony_amount': 10})
Row((354, 18, True, 354, 150580, True, 52758, True, 8222, False, 0), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 

In [7]:
query_job

QueryJob<project=degroup11, location=US, id=d483567c-0717-40c0-9467-7ad500b0797f>